In [6]:
import numpy as np
import pandas as pd
from scipy.stats import beta, bernoulli
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
import random
import math

In [18]:
def algorithm_performance():
    """
    Function that will show the performance of each algorithm we will be using in this tutorial.
    """
    
    ## calculate how many time each Ad has been choosen
    count_series = pd.Series(index_list).value_counts(normalize=True)
    print('Ad #0 has been shown', count_series[0]*100, '% of the time.')
    print('Ad #1 has been shown', count_series[1]*100, '% of the time.')
    
    print('Total Reward (Number of Clicks):', total_reward) ## print total Reward
    
    x = np.arange (0, n, 1)

    ## plot the calculated CTR for Ad #0
    data1 = go.Scatter(x=x,
                       y=ctr[0],
                       name='Calculated CTR #0',
                       line=dict(color=('rgba(10, 108, 94, .7)'),
                                 width=2))

    ## plot the line with actual CTR for Ad #0
    data2 = go.Scatter(x=[0, n],
                       y=[ACTUAL_CTR[0]] * 2,
                       name='Actual CTR #0 value',
                       line = dict(color = ('rgb(205, 12, 24)'),
                                   width = 1,
                                   dash = 'dash'))

    ## plot the calculated CTR for Ad #1
    data3 = go.Scatter(x=x,
                       y=ctr[1],
                       name='Calculated CTR #1',
                       line=dict(color=('rgba(187, 121, 24, .7)'),
                                 width=2))

    ## plot the line with actual CTR for Ad #0
    data4 = go.Scatter(x=[0, n],
                       y=[ACTUAL_CTR[1]] * 2,
                       name='Actual CTR #1 value',
                       line = dict(color = ('rgb(205, 12, 24)'),
                                   width = 1,
                                   dash = 'dash'))

    ## plot the Regret values as a function of trial number
    data5 = go.Scatter(x=x,
                       y=regret_list,
                       name='Regret')

    layout = go.Layout(title='Simulated CTR Values and Algorithm Regret',
                       xaxis={'title': 'Trial Number'},
                       yaxis1={'title': 'CTR value'},
                       yaxis2={'title': 'Regret Value'}
                       )
    fig = tools.make_subplots(rows=2, cols=1, print_grid=False, shared_yaxes=True, shared_xaxes=True)

    fig.append_trace(data1, 1, 1)
    fig.append_trace(data2, 1, 1)
    fig.append_trace(data3, 1, 1)
    fig.append_trace(data4, 1, 1)
    fig.append_trace(data5, 2, 1)

    fig['layout'].update(layout)
    iplot(fig, show_link=False)

In [7]:
# random seed 선언
RANDOM_SEED = 2022
np.random.seed(RANDOM_SEED)

In [8]:
# actual ctr 세팅
ACTUAL_CTR = [.45, .65]
print('Actual CTR for Ad #0 is:', ACTUAL_CTR[0])
print('Actual CTR for Ad #1 is:', ACTUAL_CTR[1])

Actual CTR for Ad #0 is: 0.45
Actual CTR for Ad #1 is: 0.65


# 1. Epsilon Greedy

In [9]:
e = .05 ## set the Epsilon value
n_init = 100 ## number of impressions to choose the winning Ad
impressions = [0,0]
clicks = [0,0]

In [10]:
for i in range(n_init):
    random_index = np.random.randint(0,2,1)[0]
    
    impressions[random_index] += 1
    did_click = bernoulli.rvs(ACTUAL_CTR[random_index])
    if did_click:
        clicks[random_index] += did_click
        
ctr_0 = clicks[0] / impressions[0]
ctr_1 = clicks[1] / impressions[1]
win_index = np.argmax([ctr_0, ctr_1]) ## select the Ad number with the highest CTR

In [11]:
print('After', n_init, 'initial trials Ad #', win_index, 'got the highest CTR =', round(np.max([ctr_0, ctr_1]), 2), 
      '(Real CTR value is', ACTUAL_CTR[win_index], ').'
      '\nIt will be shown', (1-e)*100, '% of the time.')

After 100 initial trials Ad # 1 got the highest CTR = 0.66 (Real CTR value is 0.65 ).
It will be shown 95.0 % of the time.


In [12]:
regret = 0 
total_reward = 0
regret_list = [] 
ctr = {0: [], 1: []}
index_list = [] 
impressions = [0,0] 
clicks = [0,0]

In [14]:
n = 1000

In [15]:
for i in range(n):    
    
    epsilon_index = random.choices([win_index, 1-win_index], [1-e, e])[0]
    index_list.append(epsilon_index)
    
    impressions[epsilon_index] += 1
    did_click = bernoulli.rvs(ACTUAL_CTR[epsilon_index])
    if did_click:
        clicks[epsilon_index] += did_click
    
    if impressions[0] == 0:
        ctr_0 = 0
    else:
        ctr_0 = clicks[0]/impressions[0]
        
    if impressions[1] == 0:
        ctr_1 = 0
    else:
        ctr_1 = clicks[1]/impressions[1]
        
    ctr[0].append(ctr_0)
    ctr[1].append(ctr_1)
    
    regret += max(ACTUAL_CTR) - ACTUAL_CTR[epsilon_index]
    regret_list.append(regret)
    total_reward += did_click

In [19]:
algorithm_performance()

Ad #0 has been shown 5.1 % of the time.
Ad #1 has been shown 94.89999999999999 % of the time.
Total Reward (Number of Clicks): 614


/Users/gsretail/Library/Python/3.8/lib/python/site-packages/plotly/tools.py:460: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



In [20]:
# 결과 값 저장
epsilon_dict = {'reward':total_reward, 
                'regret_list':regret_list, 
                'ads_count':pd.Series(index_list).value_counts(normalize=True)}